In [1]:
# set up
import json
import collections
import argparse
import random
import numpy as np

from util import *

random.seed(42)

In [28]:
# Problem 2.2
def extract_unigram_features(ex):
    """Return unigrams in the hypothesis and the premise.
    Parameters:
        ex : dict
            Keys are gold_label (int, optional), sentence1 (list), and sentence2 (list)
    Returns:
        A dictionary of BoW featurs of x.
    Example:
        "I love it", "I hate it" --> {"I":2, "it":2, "hate":1, "love":1}
    """
    # BEGIN_YOUR_CODE
    # combine sentences
    combo_sent = ex['sentence1'] + ex['sentence2']

    # count words
    bow = collections.Counter(combo_sent)
    
    return dict(bow)
    # END_YOUR_CODE

In [29]:
# Problem 2.4
def learn_predictor(train_data, valid_data, feature_extractor, learning_rate, num_epochs):
    """Running SGD on training examples using the logistic loss.
    You may want to evaluate the error on training and dev example after each epoch.
    Take a look at the functions predict and evaluate_predictor in util.py,
    which will be useful for your implementation.
    Parameters:
        train_data : [{gold_label: {0,1}, sentence1: [str], sentence2: [str]}]
        valid_data : same as train_data
        feature_extractor : function
            data (dict) --> feature vector (dict)
        learning_rate : float
        num_epochs : int
    Returns:
        weights : dict
            feature name (str) : weight (float)
    """
    # BEGIN_YOUR_CODE
    # initialize weight dict
    weights = {}
    
    for epoch in range(num_epochs): 
        for ex in train_data:
            # get BoW for each ex
            bow = feature_extractor(ex)
            
            # get label
            y = ex['gold_label']
            
            # predict prob with current weights
            y_pred = predict(weights, bow)
            
            # update weights
            for f, v in bow.items():
                # calc gradient
                gradient = (y_pred - y) * v #f_w(x^{(i)})-y^{(i)}*ϕ(x^{(i)})
                # update
                weights[f] = weights.get(f, 0) - learning_rate * gradient
            
        # calc training & validation error
        train_pred = lambda ex: 1 if predict(weights, feature_extractor(ex)) > 0.5 else 0
        train_error = evaluate_predictor([(ex, ex['gold_label']) for ex in train_data], train_pred)
        valid_error = evaluate_predictor([(ex, ex['gold_label']) for ex in valid_data], train_pred)
        print(f'Epoch {epoch + 1}: train error = {train_error:.3f}, valid error = {valid_error:.3f}')
            
    return weights
    # END_YOUR_CODE

In [30]:
def test_unigram():
    train_data = read_dataset('data/train.json', -1)
    valid_data = read_dataset('data/dev.json', -1)
    feature_extractor = extract_unigram_features
    weights = learn_predictor(train_data, valid_data, feature_extractor, 0.01, 10)
    predictor = lambda ex: 1 if dot(weights, feature_extractor(ex)) > 0 else 0
    train_err = evaluate_predictor([(ex, ex['gold_label']) for ex in train_data], predictor)
    valid_err = evaluate_predictor([(ex, ex['gold_label']) for ex in valid_data], predictor)
    print('train error={}, valid error={}'.format(train_err, valid_err))
    error_analysis(valid_data[:100], feature_extractor, weights, 'error_analysis_unigram.txt')

test_unigram()

Load 10000 examples from data/train.json
Load 10000 examples from data/dev.json
Epoch 1: train error = 0.352, valid error = 0.370
Epoch 2: train error = 0.324, valid error = 0.364
Epoch 3: train error = 0.306, valid error = 0.361
Epoch 4: train error = 0.294, valid error = 0.361
Epoch 5: train error = 0.283, valid error = 0.361
Epoch 6: train error = 0.272, valid error = 0.362
Epoch 7: train error = 0.267, valid error = 0.360
Epoch 8: train error = 0.262, valid error = 0.359
Epoch 9: train error = 0.256, valid error = 0.359
Epoch 10: train error = 0.251, valid error = 0.360
train error=0.251, valid error=0.36


In [31]:
# Problem 2.6
def extract_custom_features(ex):
    """Design your own features.
    """
    # BEGIN_YOUR_CODE
    # initialize feature dict (bow in unigram)
    features = {}
    
    sentence1 = ex['sentence1']
    sentence2 = ex['sentence2']
    
    # unigram with all lowercase
    all_words = sentence1 + sentence2
    for word in set(all_words):
        features[f'unigram_{word.lower()}'] = all_words.count(word)
    
    # bigram
    def get_bigrams(sentence):
        return [f"{sentence[i]}_{sentence[i+1]}" for i in range(len(sentence)-1)]
    
    bigrams1 = get_bigrams(sentence1)
    bigrams2 = get_bigrams(sentence2)
    all_bigrams = bigrams1 + bigrams2
    for bigram in set(all_bigrams):
        features[f'bigram_{bigram.lower()}'] = all_bigrams.count(bigram)
    
    # negation
    negation_words = set(['not', 'no', 'never', 'neither', 'nor', 'none'])
    features['negation_s1'] = sum(1 for word in sentence1 if word.lower() in negation_words)
    features['negation_s2'] = sum(1 for word in sentence2 if word.lower() in negation_words)
    
    return features
    # END_YOUR_CODE

In [32]:
## performs better on validation (dev) than unigram
def test_custom():
    train_data = read_dataset('data/train.json', -1)
    valid_data = read_dataset('data/dev.json', -1)
    feature_extractor = extract_custom_features
    weights = learn_predictor(train_data, valid_data, feature_extractor, 0.01, 10)
    predictor = lambda ex: 1 if dot(weights, feature_extractor(ex)) > 0 else 0
    train_err = evaluate_predictor([(ex, ex['gold_label']) for ex in train_data], predictor)
    valid_err = evaluate_predictor([(ex, ex['gold_label']) for ex in valid_data], predictor)
    print('train error={}, valid error={}'.format(train_err, valid_err))
    error_analysis(valid_data[:100], feature_extractor, weights, 'error_analysis_custom.txt')
    
test_custom()

Load 10000 examples from data/train.json
Load 10000 examples from data/dev.json
Epoch 1: train error = 0.297, valid error = 0.367
Epoch 2: train error = 0.243, valid error = 0.358
Epoch 3: train error = 0.210, valid error = 0.353
Epoch 4: train error = 0.188, valid error = 0.351
Epoch 5: train error = 0.169, valid error = 0.349
Epoch 6: train error = 0.151, valid error = 0.350
Epoch 7: train error = 0.138, valid error = 0.350
Epoch 8: train error = 0.127, valid error = 0.350
Epoch 9: train error = 0.115, valid error = 0.350
Epoch 10: train error = 0.105, valid error = 0.351
train error=0.1055, valid error=0.3514


In [33]:
# Problem 2.8
def extract_unigram_features(ex):
    """Return unigrams in the hypothesis and the premise.
    Parameters:
        ex : dict
            Keys are gold_label (int, optional), sentence1 (list), and sentence2 (list)
    Returns:
        A dictionary of BoW featurs of x.
    Example:
        "I love it", "I hate it" --> {"I":2, "it":2, "hate":1, "love":1}
    """
    # BEGIN_YOUR_CODE
    # combine sentences
    sent2 = ex['sentence2']

    # count words
    bow = collections.Counter(sent2)
    
    return dict(bow)
    # END_YOUR_CODE
    
    
def test_unigram():
    train_data = read_dataset('data/train.json', -1)
    valid_data = read_dataset('data/dev.json', -1)
    feature_extractor = extract_unigram_features
    weights = learn_predictor(train_data, valid_data, feature_extractor, 0.01, 10)
    predictor = lambda ex: 1 if dot(weights, feature_extractor(ex)) > 0 else 0
    train_err = evaluate_predictor([(ex, ex['gold_label']) for ex in train_data], predictor)
    valid_err = evaluate_predictor([(ex, ex['gold_label']) for ex in valid_data], predictor)
    print('train error={}, valid error={}'.format(train_err, valid_err))
    error_analysis(valid_data[:100], feature_extractor, weights, 'error_analysis_unigram.txt')

test_unigram()

Load 10000 examples from data/train.json
Load 10000 examples from data/dev.json
Epoch 1: train error = 0.318, valid error = 0.315
Epoch 2: train error = 0.298, valid error = 0.301
Epoch 3: train error = 0.288, valid error = 0.295
Epoch 4: train error = 0.280, valid error = 0.295
Epoch 5: train error = 0.273, valid error = 0.292
Epoch 6: train error = 0.266, valid error = 0.289
Epoch 7: train error = 0.260, valid error = 0.287
Epoch 8: train error = 0.255, valid error = 0.285
Epoch 9: train error = 0.251, valid error = 0.282
Epoch 10: train error = 0.247, valid error = 0.281
train error=0.2468, valid error=0.2805


In [2]:
# Problem 3.1
def count_cooccur_matrix(tokens, window_size=4):
    """Compute the co-occurrence matrix given a sequence of tokens.
    For each word, n words before and n words after it are its co-occurring neighbors.
    For example, given the tokens "in for a penny , in for a pound",
    the neighbors of "penny" given a window size of 2 are "for", "a", ",", "in".
    Parameters:
        tokens : [str]
        window_size : int
    Returns:
        word2ind : dict
            word (str) : index (int)
        co_mat : np.array
            co_mat[i][j] should contain the co-occurrence counts of the words indexed by i 
            and j according to the dictionary word2ind.
    """
    # BEGIN_YOUR_CODE
    # dict for words & corresponding indices (w=word, i=index)
    word2ind = {w: i for i, w in enumerate(set(tokens))}
    
    # initialize co-occurence matrix
    vocab_len = len(word2ind)
    co_mat = np.zeros((vocab_len, vocab_len), dtype = int)
    
    for i, t in enumerate(tokens):
        token_ind = word2ind[t] #gets index of current token
        
        # window range
        start = max(0, i - window_size)
        end = min(len(tokens), i + window_size + 1)
        
        # count co-occurrences
        for n in range(start, end):
            if i != n: #skip current token
                neighbor = tokens[n]
                neighbor_ind = word2ind[neighbor]
                co_mat[token_ind][neighbor_ind] += 1
                
    return word2ind, co_mat
    # END_YOUR_CODE

In [7]:
# Problem 3.3
def cooccur_to_embedding(co_mat, embed_size=50):
    """Convert the co-occurrence matrix to word embedding using truncated SVD. Use the np.linalg.svd function.
    Parameters:
        co_mat : np.array
            vocab size x vocab size
        embed_size : int
    Returns:
        embeddings : np.array
            vocab_size x embed_size
    """
    # BEGIN_YOUR_CODE
    '''# apply log transformation to co_mat
    log_co_mat = np.log(co_mat + 1) #add 1 to prevent log(0)
    
    # truncated SVD - truncate based on embed_size
    U, S, Vt = np.linalg.svd(log_co_mat, full_matrices = False, hermitian = False) #set full_matrices=False since 2d vector after log transformation
    U, S, Vt = np.linalg.svd(co_mat, hermitian = False)
    
    U_trunc = U[:, :embed_size]
    S_trunc = S[:embed_size]

    # calc embeddings
    embeddings = U_trunc * np.sqrt(S_trunc)'''
    U, s, Vt = np.linalg.svd(co_mat, full_matrices=False)
    
    # truncate to the specified embedding size
    U_truncated = U[:, :embed_size]
    s_truncated = s[:embed_size]
    
    # compute the embeddings
    embeddings = -U_truncated * s_truncated[np.newaxis, :]
            
    return embeddings
    # END_YOUR_CODE

In [8]:
# Problem 3.4
def top_k_similar(word_ind, embeddings, word2ind, k=10, metric='dot'):
    """Return the top k most similar words to the given word (excluding itself).
    You will implement two similarity functions.
    If metric='dot', use the dot product.
    If metric='cosine', use the cosine similarity.
    Parameters:
        word_ind : int
            index of the word (for which we will find the similar words)
        embeddings : np.array
            vocab_size x embed_size
        word2ind : dict
        k : int
            number of words to return (excluding self)
        metric : 'dot' or 'cosine'
    Returns:
        topk_words : [str]
    """
    # BEGIN_YOUR_CODE
    # get embedding of target word
    target_embed = embeddings[word_ind]
    
    # compute similarities
    if metric == 'dot':
        similarities = np.dot(embeddings, target_embed)
    else: #place holder for 'cosine'
        print('Cosine similarity selected')
        pass
    
    # adjust similarity btwn word & itself to be very low
    similarities[word_ind] = float('-inf')
    
    # get indices of top k similar words
    top_k_inds = np.argsort(similarities)[-k:][::-1]
    
    # get words associated with indices (w=word, i=index)
    ind2word = {i: w for w, i in word2ind.items()}
    topk_words = [ind2word[i] for i in top_k_inds]
    
    return topk_words
    # END_YOUR_CODE

In [9]:
# Problem 3.5
def test_embedding(words=['man', 'woman', 'happy', 'sad', 'emma', 'knightley']):
    tokens = read_corpus()
    word2ind, co_mat = count_cooccur_matrix(tokens, window_size=1)
    embeddings = cooccur_to_embedding(co_mat, 25)
    for word in words:
        word_ind = word2ind[word]
        top_k_words = top_k_similar(word_ind, embeddings, word2ind, k=10, metric='dot')
        print('top k most similar words to', word)
        print(' '.join(top_k_words))
    return embeddings

test2 = test_embedding()

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sophiajuco/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


top k most similar words to man
and i it the " in she but that ,
top k most similar words to woman
and , the i it in " of to was
top k most similar words to happy
, and the mr i be it to her a
top k most similar words to sad
of and in , . was very to is with
top k most similar words to emma
and i " , it she mr he the you
top k most similar words to knightley
mr mrs i and it she " weston elton he


In [17]:
test1

array([[-1.32778902, -0.49065546,  0.05708003, ..., -0.39414713,
        -1.44234663, -0.39142897],
       [-0.45656484, -0.24753939, -0.25098383, ...,  0.08277396,
        -0.14086778,  0.3123277 ],
       [-0.12532598,  0.05601888,  0.02379837, ...,  0.08075457,
        -0.09662852,  0.14346088],
       ...,
       [-0.94835287, -1.07232428,  0.75208154, ...,  0.1094477 ,
         0.07309543, -0.21885018],
       [-0.32009247, -0.03828584, -0.33083983, ..., -0.01092303,
         0.06792273,  0.65760978],
       [-1.16046274,  0.04440773, -0.06263916, ..., -0.10973086,
         0.17192602,  0.32812246]])

In [10]:
test2

array([[ 1.32778902,  0.49065546, -0.05708003, ...,  0.39414713,
         1.44234663,  0.39142897],
       [ 0.45656484,  0.24753939,  0.25098383, ..., -0.08277396,
         0.14086778, -0.3123277 ],
       [ 0.12532598, -0.05601888, -0.02379837, ..., -0.08075457,
         0.09662852, -0.14346088],
       ...,
       [ 0.94835287,  1.07232428, -0.75208154, ..., -0.1094477 ,
        -0.07309543,  0.21885018],
       [ 0.32009247,  0.03828584,  0.33083983, ...,  0.01092303,
        -0.06792273, -0.65760978],
       [ 1.16046274, -0.04440773,  0.06263916, ...,  0.10973086,
        -0.17192602, -0.32812246]])

In [19]:
np.sign(test1[:][1])

array([-1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1.,
       -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.])

In [13]:
test

array([[-0.04036213,  0.05763836, -0.00996007, ...,  0.00808308,
         0.02513569,  0.01602217],
       [-0.03936486,  0.03765437,  0.00375551, ..., -0.00895273,
        -0.02419763, -0.06151337],
       [-0.00661321,  0.00558734, -0.00437265, ..., -0.00838526,
        -0.00736296,  0.00481276],
       ...,
       [-0.1479234 ,  0.09023045,  0.25375   , ...,  0.01438565,
         0.04689914,  0.05125293],
       [-0.01197331,  0.00372352,  0.0127503 , ...,  0.01344636,
         0.02508408, -0.02837081],
       [-0.06456793,  0.04396606,  0.01948028, ..., -0.02215805,
        -0.0089393 , -0.00366436]])

In [6]:
# Problem 3.6
def top_k_similar(word_ind, embeddings, word2ind, k=10, metric='dot'):
    """Return the top k most similar words to the given word (excluding itself).
    You will implement two similarity functions.
    If metric='dot', use the dot product.
    If metric='cosine', use the cosine similarity.
    Parameters:
        word_ind : int
            index of the word (for which we will find the similar words)
        embeddings : np.array
            vocab_size x embed_size
        word2ind : dict
        k : int
            number of words to return (excluding self)
        metric : 'dot' or 'cosine'
    Returns:
        topk_words : [str]
    """
    # BEGIN_YOUR_CODE
    # get embedding of target word
    target_embed = embeddings[word_ind]
    
    # compute similarities
    if metric == 'dot':
        similarities = np.dot(embeddings, target_embed)
    elif metric == 'cosine':
        norm_target = np.linalg.norm(target_embed)
        norm_embeds = np.linalg.norm(embeddings, axis=1)
        similarities = np.dot(embeddings, target_embed) / (norm_embeds * norm_target)
    else: #error if neither dot or cosine
        raise ValueError("Metric must be either 'dot' or 'cosine'")
    
    # adjust similarity btwn word & itself to be very low
    similarities[word_ind] = float('-inf')
    
    # get indices of top k similar words
    top_k_inds = np.argsort(similarities)[-k:][::-1]
    
    # get words associated with indices (w=word, i=index)
    ind2word = {i: w for w, i in word2ind.items()}
    topk_words = [ind2word[i] for i in top_k_inds]
    
    return topk_words
    # END_YOUR_CODE

In [7]:
# Problem 3.7
def test_embedding(words=['man', 'woman', 'happy', 'sad', 'emma', 'knightley']):
    tokens = read_corpus()
    word2ind, co_mat = count_cooccur_matrix(tokens, window_size=1)
    embeddings = cooccur_to_embedding(co_mat, 100)
    for word in words:
        word_ind = word2ind[word]
        top_k_words = top_k_similar(word_ind, embeddings, word2ind, k=10, metric='cosine')
        print('top k most similar words to', word)
        print(' '.join(top_k_words))

test_embedding()

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sophiajuco/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


top k most similar words to man
woman lady girl creature _man_ gentleman moment person tis farmer
top k most similar words to woman
man lady gentleman person _man_ tis people girl creature circumstance
top k most similar words to happy
obliging ready agreeable serious busy limited kind complete distressing pleasant
top k most similar words to sad
narrow quivering ninth lame painful delightful small lucky few striking
top k most similar words to emma
harriet jane mr isabella mrs frank he then knightley ,'"
top k most similar words to knightley
elton weston perry churchill woodhouse martin isabella fairfax goddard dixon
